In [1]:
#Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from scipy.stats import linregress
import seaborn as sns
import os
import gmaps

from config import key

In [2]:
weather_df = pd.read_csv("../cities_weather.csv")
weather_df

,Unnamed: 0,City,Country,Date,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,pevek,RU,1619032925,69.7008,170.3133,-6.24,68,12,7.97
1,1,esperance,AU,1619032655,-33.8667,121.9000,14.00,94,90,1.54
2,2,butaritari,KI,1619032928,3.0707,172.7902,27.83,78,100,7.96
3,3,puerto ayora,EC,1619032817,-0.7393,-90.3518,27.00,78,75,5.14
4,4,arraial do cabo,BR,1619032931,-22.9661,-42.0278,23.00,83,40,4.12
...,...,...,...,...,...,...,...,...,...,...
540,540,yakeshi,CN,1619036591,49.2833,120.7333,11.37,53,100,2.85
541,541,rundu,NaN,1619036592,-17.9333,19.7667,17.68,48,2,2.82
542,542,havelock,US,1619036594,34.8791,-76.9013,24.44,53,20,8.75
543,543,sakakah,SA,1619036595,29.9697,40.2064,31.00,12,0,5.14


In [3]:
gmaps.configure(api_key=key)

In [4]:
locations = weather_df[["Lat","Long"]]
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=5)

fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness less than or equal to 10, and max temp between 70 and 80
narrowed_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & 
                                  (weather_df["Cloudiness"] < 10) & 
                                  (weather_df["Max Temp"] >= 21)&
                                  (weather_df["Max Temp"] <= 27)]
narrowed_city_df.head()

,Unnamed: 0,City,Country,Date,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
16,16,chuy,UY,1619032930,-33.6971,-53.4616,21.82,63,5,7.45
28,28,rikitea,PF,1619032968,-23.1203,-134.9692,25.68,72,3,5.76
94,94,victoria,HK,1619032433,22.2855,114.1577,23.33,82,0,2.85
107,107,cockburn town,TC,1619033101,21.4612,-71.1419,26.09,75,7,7.47
114,114,roebourne,AU,1619033112,-20.7833,117.1333,25.84,46,0,3.37


In [9]:
#Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Long"]]

#Add hotel name column to df
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Long,Hotel Name
16,chuy,UY,-33.6971,-53.4616,
28,rikitea,PF,-23.1203,-134.9692,
94,victoria,HK,22.2855,114.1577,
107,cockburn town,TC,21.4612,-71.1419,
114,roebourne,AU,-20.7833,117.1333,


In [11]:
#Build base url and state parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : key}

#Iterate through hotel_df to retrieve city name, lat, long and country
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Long"]
    city_name = row["City"]
    country = row["Country"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Gathering Results for Index {index}: {city_name, country}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # Wait 1 sec to make api request to avoid SSL Error
    time.sleep(1)
    
    # save the hotel name to hotel_df
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print("--------------------------------")
        
    # show missing fields
    except (KeyError, IndexError):
        print("Missing field/result - skipping.")
        print("--------------------------------")
        
# Print end of search once searching is completed
print("-------End of Search-------")

Gathering Results for Index 16: ('chuy', 'UY').
Closest hotel in chuy is Descanso al Paso Chuy.
--------------------------------
Gathering Results for Index 28: ('rikitea', 'PF').
Missing field/result - skipping.
--------------------------------
Gathering Results for Index 94: ('victoria', 'HK').
Closest hotel in victoria is The Nap @ K11 Tsim Sha Tsui.
--------------------------------
Gathering Results for Index 107: ('cockburn town', 'TC').
Closest hotel in cockburn town is Osprey Beach Hotel.
--------------------------------
Gathering Results for Index 114: ('roebourne', 'AU').
Closest hotel in roebourne is Latitude20 Roebourne Village.
--------------------------------
Gathering Results for Index 135: ('beloha', 'MG').
Missing field/result - skipping.
--------------------------------
Gathering Results for Index 197: ('laguna', 'US').
Closest hotel in laguna is Hilton Garden Inn Sacramento Elk Grove.
--------------------------------
Gathering Results for Index 217: ('sao goncalo do s

In [15]:
hotel_df

,City,Country,Lat,Long,Hotel Name
16,chuy,UY,-33.6971,-53.4616,Descanso al Paso Chuy
28,rikitea,PF,-23.1203,-134.9692,
94,victoria,HK,22.2855,114.1577,The Nap @ K11 Tsim Sha Tsui
107,cockburn town,TC,21.4612,-71.1419,Osprey Beach Hotel
114,roebourne,AU,-20.7833,117.1333,Latitude20 Roebourne Village
135,beloha,MG,-25.1667,45.0500,
197,laguna,US,38.4210,-121.4238,Hilton Garden Inn Sacramento Elk Grove
217,sao goncalo do sapucai,BR,-21.8922,-45.5953,Novo Hotel
221,myitkyina,MM,25.3833,97.4000,Palm Spring Resort Hotel Myitkyina
247,mineiros,BR,-17.5694,-52.5511,Confort Prime Hotel


In [16]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [17]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))